### データの前処理

In [9]:
# import pandas as pd
# import os

# path = "data"

# for filename in os.listdir(path):
#     file_path = os.path.join(path, filename)
#     # CSVファイル以外の場合は削除する
#     if not filename.endswith('.csv'):
#         os.remove(file_path)

In [10]:
# path = "data/tani"

# for filename in os.listdir(path):
#     file_path = os.path.join(path, filename)
#     # CSVファイル以外の場合は削除する
#     df = pd.read_csv(file_path,skiprows=15,index_col=0)
#     df = df.drop("Units:")
#     df.to_csv(file_path)

モーションキャプチャとフォースプレートのデータの分割

In [11]:
# import os
# import pandas as pd

# # Specify the directory you want to use
# directory = 'data/sasaki'
# save1 = 'data/sasaki/motion'
# save2 = 'data/sasaki/force'

# # Loop over all files in the directory
# for filename in os.listdir(directory):
#     # Only process CSV files
#     if filename.endswith('.csv'):
#         filepath = os.path.join(directory, filename)

#         # Read the file line by line
#         with open(filepath, 'r') as f:
#             lines = f.readlines()

#         # Initialize empty lists to hold the data
#         data1 = []
#         data2 = []

#         # Use a flag to indicate which dataset we are currently populating
#         current_dataset = 1

#         # Iterate over the lines in the file
#         for line in lines:
#             # If the line contains "FORCE PLATES", switch to the second dataset
#             if "FORCE PLATES" in line:
#                 current_dataset = 2
#                 continue

#             # Ignore empty lines
#             if line.strip() == '':
#                 continue

#             # Split the line into fields
#             fields = line.split(",")
            
#             # Add the fields to the appropriate dataset
#             if current_dataset == 1:
#                 data1.append(fields)
#             else:
#                 data2.append(fields)

#         # Convert the lists to pandas DataFrames
#         data1 = pd.DataFrame(data1[9:])  # Skip the first 9 rows
#         data2 = pd.DataFrame(data2[6:])  # Skip the first 6 rows

#         # Write the two DataFrames to CSV files
#         data1.to_csv(save1 + filename, index=False, header=None)
#         data2.to_csv(save2 + filename, index=False, header=None)

動画ファイルのリネーム

In [12]:
# import os
# from natsort import natsorted

# def rename_files_in_folder(folder_path, start_number,angle):
#     files = natsorted(os.listdir(folder_path))  # Sort the files in ascending order
#     for filename in files:
#         if os.path.isfile(os.path.join(folder_path, filename)):  # Only process files, not directories
#             extension = os.path.splitext(filename)[1]  # Get the file extension
#             os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, angle + "_" + str(start_number) + extension))
#             start_number += 1

# # Usage
# angle = "front"
# rename_files_in_folder(f'/mnt/d/gakuin/videos/sasaki/{angle}', 101,angle)  # Starting from 100

In [13]:
import numpy as np
import pandas as pd
from tabulate import tabulate

def calculate_delay_frame(csv_file1, csv_file2, name):
    df1 = pd.read_csv(csv_file1, index_col="frame")
    df2 = pd.read_csv(csv_file2, index_col="frame")

    df1.interpolate(method="linear", limit_direction="both", inplace=True)
    df2.interpolate(method="linear", limit_direction="both", inplace=True)

    df1_y = df1[f"{name}_y"] - df1[f"{name}_y"].mean()
    df2_y = df2[f"{name}_y"] - df2[f"{name}_y"].mean()

    corr = np.correlate(df1_y, df2_y, "full")
    delay_frame = int(corr.argmax() - (len(df2_y) - 1))

    return delay_frame

def delay_frame_viz(csv_file1, csv_file2, delay_frame):
    if delay_frame > 0:
        table = [[csv_file1, 1 + delay_frame], [csv_file2, 1]]
    else:
        table = [[csv_file1, 1], [csv_file2, 1 - delay_frame]]
    print(tabulate(table, headers=["ビデオ", "フレーム"], tablefmt="grid"))

# パラメーター
name = "LEFT_SHOULDER"  # 実際の変数名に置き換えてください

# ファイルをループで処理
for file in range(31, 68):
    csv_file1 = f"/mnt/d/gakuin/documents/tani/front_{100+file}/position_data.csv"
    csv_file2 = f"/mnt/d/gakuin/documents/tani/side_{100+file}/position_data.csv"
    delay_frame = calculate_delay_frame(csv_file1, csv_file2, name)
    delay_frame_viz(csv_file1, csv_file2, delay_frame)


+----------------------------------------------------------+------------+
| ビデオ                                                   |   フレーム |
+==========================================================+============+
| /mnt/d/gakuin/documents/tani/front_131/position_data.csv |         23 |
+----------------------------------------------------------+------------+
| /mnt/d/gakuin/documents/tani/side_131/position_data.csv  |          1 |
+----------------------------------------------------------+------------+
+----------------------------------------------------------+------------+
| ビデオ                                                   |   フレーム |
+==========================================================+============+
| /mnt/d/gakuin/documents/tani/front_132/position_data.csv |         52 |
+----------------------------------------------------------+------------+
| /mnt/d/gakuin/documents/tani/side_132/position_data.csv  |          1 |
+---------------------------------------------------

contact_plateから3d_position_dataのインデックスを取得

In [14]:
import pandas as pd

df = pd.read_csv("/mnt/d/gakuin/documents/tani/front_131/3d_position_data.csv", index_col="frame")
df

,NOSE_x,NOSE_y,NOSE_z,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_z,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_z,LEFT_EYE_OUTER_x,...,LEFT_HEEL_z,RIGHT_HEEL_x,RIGHT_HEEL_y,RIGHT_HEEL_z,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_z,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_z
frame,,,,,,,,,,,,,,,,,,,,,
1,1.381277,1.477104,1.129042,1.362749,1.518317,1.141768,1.349863,1.520961,1.146483,1.338227,...,1.417926,1.510450,0.038853,1.457035,1.277064,0.001676,1.238921,1.614194,0.017122,1.255474
2,1.380712,1.477547,1.126452,1.362777,1.518829,1.137731,1.349915,1.521470,1.142014,1.338314,...,1.416669,1.510268,0.040398,1.456498,1.276675,0.004122,1.237085,1.614659,0.017378,1.254738
3,1.381190,1.475368,1.126506,1.363796,1.517467,1.137154,1.350907,1.520257,1.141001,1.339387,...,1.419503,1.510410,0.041789,1.456484,1.276663,0.005274,1.236236,1.614696,0.017308,1.254827
4,1.380499,1.473874,1.123827,1.363389,1.516672,1.133486,1.350595,1.519692,1.137812,1.338830,...,1.420394,1.510500,0.042134,1.456561,1.276523,0.006494,1.235683,1.614520,0.017341,1.254634
5,1.377612,1.473907,1.123237,1.360973,1.516924,1.132445,1.348829,1.519880,1.136858,1.336169,...,1.423779,1.510777,0.042914,1.456845,1.276174,0.008167,1.235251,1.614146,0.017371,1.254393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3683,1.456372,1.528772,1.010817,1.441005,1.568267,1.025610,1.429092,1.570162,1.030831,1.415105,...,1.276894,1.669435,0.035218,1.258902,1.293164,0.024240,1.081670,1.722929,0.021559,1.061392
3684,1.455096,1.528939,1.010590,1.440536,1.568610,1.025563,1.428450,1.570543,1.030782,1.414624,...,1.276799,1.669537,0.035166,1.258847,1.293114,0.024333,1.080962,1.722948,0.021550,1.061018
3685,1.454954,1.529165,1.010955,1.440431,1.568910,1.025773,1.428326,1.570894,1.030955,1.414482,...,1.276820,1.669527,0.035148,1.258779,1.293095,0.024325,1.081229,1.723094,0.021519,1.060973


フォースプレートデータと3d_position_dataの同期

In [15]:
import pandas as pd
import sys
sys.path.append('py')
from utils import concat_data,synchronize_data,process_dataframe

#フォースプレートのデータと骨格座標データを読み込む
def read_data(number:int) -> pd.DataFrame:
    df_forceplate = pd.read_csv(f"data/tani/forceplate/{number}.csv",index_col=0)
    df_forceplate.index = df_forceplate.index.astype(int)
    df_forceplate = df_forceplate[["Force:X", "Force:Y", "Force:Z"]]
    df_3d = pd.read_csv(f'/mnt/d/gakuin/documents/tani/side_{number}/3d_position_data.csv',index_col=0)
    return df_3d,df_forceplate

#骨格座標データにおける接地フレームを取得する
def contact_frame(n:int)->int:
    df_contact = pd.read_csv("contact_plate.csv",index_col=0)
    contact_frame = df_contact.loc[f'side_{n}',"frame"]
    return int(contact_frame)


#フォースプレートのデータにおける接地フレームを取得する
def detect_contact_frame(number:int)->int:
    df = pd.read_csv(f'data/tani/forceplate/{number}.csv',index_col=0)
    df = df.drop("Units:")
    df = df[['Force:Y','Force:Z']].astype(float)
    df = df.loc[df["Force:Z"] > abs(float(0))]
    df = process_dataframe(df)
    return int(df.index[0]-1)


def process_dataframe(df):
    # # Set 'Sample #' as the index of the DataFrame
    df.index = df.index.astype(int)

    # Calculate the difference between consecutive indices
    df['Index_Diff'] = df.index.to_series().diff()

    # Identify where the difference is not 1 (indicating a gap in the sequence)
    gap_indices = df.index[df['Index_Diff'] != 1].tolist()

    # Add the start and end indices for completeness
    gap_indices = [df.index.min() - 1] + gap_indices + [df.index.max() + 1]

    # Initialize a list to store the slices of the DataFrame that meet the condition
    df_list = []

    # Loop over the gaps and slice the DataFrame
    for start, end in zip(gap_indices[:-1], gap_indices[1:]):
        df_slice = df.loc[start+1:end-1]

        # If the slice has 10 or more rows, add it to the list
        if len(df_slice) >= 10:
            df_list.append(df_slice)

    # Concatenate all the slices into a single DataFrame
    df_result = pd.concat(df_list)

    # Find the gaps in the index sequence
    gap_starts = df.index[df['Index_Diff'] > 1]
    
    return df_result

In [16]:

import pandas as pd

#フォースプレートのデータと骨格座標データを結合する
def concat_data(df1:pd.DataFrame,df2:pd.DataFrame,std_frame1:int,std_frame2:int,cam1_fps:int,cam2_fps:int) -> pd.DataFrame:
    '''
    df1:骨格座標データ
    df2:フォースプレートデータ
    std_frame1:正面動画の接地フレーム
    std_frame2:横動画の接地フレーム
    cam1_fps:正面動画のfps
    cam2_fps:横動画のfps
    '''
    n = min(len(df1),len(df2))
    df = pd.DataFrame()
    for i in range(n):
        frame1 = df1.index[i]
        frame2 = synchronize_data(std_frame1,std_frame2,cam1_fps,cam2_fps,frame1)
        try:
            df_1 = df1[df1.index==df1.index[i]]
            df_2 = df2[df2.index==frame2].set_index(df_1.index,drop=True)
            df = pd.concat([df,pd.concat([df_1,df_2],axis=1)],axis=0)
        except:
            print('pass')
            pass
    return df

#異なるfpsの動画のフレームを同期させる
def synchronize_data(std_frame1:int,std_frame2:int,cam1_fps:int,cam2_fps:int,frame1:int) -> int:
    frame2 = (frame1 - std_frame1) * cam2_fps / cam1_fps + std_frame2
    return int(round(frame2+0.1))

#フォースプレートのデータと骨格座標データを読み込む
def read_data(number:int) -> pd.DataFrame:
    df_forceplate = pd.read_csv(f"data/tani/forceplate/{number}.csv",index_col=0)
    df_forceplate.index = df_forceplate.index.astype(int)
    df_forceplate = df_forceplate[["Force:X", "Force:Y", "Force:Z"]]
    
    df_3d = pd.read_csv(f'/mnt/d/gakuin/documents/tani/side_{number}/3d_position_data.csv',index_col=0)

    return df_3d,df_forceplate

#骨格座標データにおける接地フレームを取得する
def contact_frame(n:int)->int:
    df_contact = pd.read_csv("csv/contact_plate.csv",index_col=0)
    contact_frame = df_contact.loc[f'side_{n}',"frame"]
    return int(contact_frame)

# def syncro_frame(number:int) -> int:
#     df_syncro = pd.read_csv("syncro_frames.csv",index_col=0)
#     std_frame = df_syncro.loc[number].tolist()
#     if std_frame[0] < std_frame[1]:
#         return contact_frame(number)
#     else:
#         print("error")
#         a= contact_frame(number)
#         return synchronize_data(std_frame[1],std_frame[0],960,960,a)

# #正面動画と横動画の同期フレームを取得する
# def syncro_frame(number:int) -> int:
#     df_syncro = pd.read_csv("syncro_frames.csv",index_col=0)
#     syncro_frame = int(df_syncro.loc[number,'side'])
#     return syncro_frame


#フォースプレートのデータにおける接地フレームを取得する
def detect_contact_frame(number:int)->int:
    df = pd.read_csv(f'data/tani/forceplate/{number}.csv',index_col=0)
    df = df[['Force:Y','Force:Z']].astype(float)
    df = df.loc[df["Force:Z"] > abs(float(0))]
    df = process_dataframe(df)
    return int(df.index[0]-1)

def syncro_frames()->pd.DataFrame:
    df_syncro = pd.read_csv('csv/syncro_frames.csv')
    return df_syncro

In [17]:
df_syncro = pd.read_csv('csv/syncro_frames.csv',index_col=0)
df_syncro

,front,side,fps_front,fps_side,Unnamed: 5
101,1,308,960,960,NaN
102,1,130,960,960,NaN
103,1,296,960,960,NaN
104,1,299,960,960,NaN
105,23,1,960,960,NaN
106,1,435,960,960,NaN
107,1,475,960,960,NaN
108,258,1,960,960,NaN
109,772,2444,960,960,NaN
110,2240,2283,960,960,NaN


同期データの設定

In [18]:

import os
import pandas as pd

for n in range(131,168):
    print(n)
    df1,df2 = read_data(n)
    df2['second'] = [i/1000 for i in range(len(df2))]
    std_frame1 = contact_frame(n)
    std_frame2 = detect_contact_frame(n)
    cam1_fps = int(df_syncro.loc[df_syncro.index==n,'fps_side'])
    cam2_fps = 1000
    save_path = 'data/concat_data'
    os.makedirs(save_path,exist_ok=True)
    print(std_frame1,std_frame2)
    df = concat_data(df1,df2,std_frame1,std_frame2,cam1_fps,cam2_fps)
    df = df.astype(float)
    df = df.loc[df["Force:Z"] > abs(float(0))]
    df.to_csv(f"{save_path}/concat_data_{n}.csv")

131
2086 2014
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass


132
149 8802
133
2086 5320
134
1934 5195
135
2457 5349
136
2101 5747
137
2097 5843
138
1630 5806
139
2142 5740
140
2434 6081
141
2171 6211
142
1432 5480
143
2215 5892
144
544 4818
145
2945 6697
146
2068 5897
147
2043 6472
148
1635 5629
149
1572 5179
150
1432 5447
151
1419 6054
152
936 5426
153
1454 5334
154
1371 5683
155
1399 5466
156
1488 5231
157
1193 5288
158
3113 7218
159
1120 5017
160
1349 5452
161
1240 5050
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
162
407 3537
163
392 3701
164
382 2150
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
165
329 4205
166
520 3527
pass
pas

フォースプレートの接地する瞬間の特定

In [19]:
# import pandas as pd
# import numpy as np

# data_path = "data/tani/forceplate/149.csv"
# df = pd.read_csv(data_path,index_col=0)
# df.rename(columns={'Moment:Z\n': 'Moment:Z.1'}, inplace=True)
# df = df[['Force:X.1', 'Force:Y.1', 'Force:Z', 'Moment:X.1', 'Moment:Y.1', 'Moment:Z.1']]
# df = df.astype(float)
# df[7921:7925]['Force:Z'].plot()

地面反力及びトルクの計算

In [20]:
# columns = ["Force:X", "Force:Y", "Force:Z", "Moment:X", "Moment:Y", "Moment:Z","Force:X.1", "Force:Y.1", "Force:Z.1", "Moment:X.1", "Moment:Y.1", "Moment:Z.1"]
# df = df[columns]
# df["Moment:Z.1"] = df["Moment:Z.1"].astype(float)
# df

In [21]:
# df1 = df.iloc[:,:6]

# df1 = df1.apply(pd.to_numeric, errors='coerce')
# df1 = df1.loc[df1["Force:Z.1"] > abs(float(0))]

# df1

In [22]:
# df1 = process_dataframe(df1)
# df1

加速度の算出